In [3]:
import numpy as np
from fastembed import TextEmbedding

In [4]:
MODEL = 'jinaai/jina-embeddings-v2-small-en'

embedding_model = TextEmbedding(MODEL)

## 1. Embedding the query into vector space representation

In [5]:
sample_query = 'I just discovered the course. Can I join now?'

documents = [sample_query]

In [6]:
embedding_generator = embedding_model.embed(documents)

results = list(embedding_generator)

In [15]:
print(len(results))
print(len(results[0]))
print(results)

1
512
[array([-7.63946373e-02, -7.30555514e-02,  5.86501632e-02,  3.92670711e-02,
       -1.41796944e-02, -4.68342392e-02,  2.69862839e-02,  3.49039298e-02,
        1.53419599e-03, -1.10407681e-02,  5.57556382e-02, -2.88360335e-02,
       -4.87239656e-02, -9.90923940e-02,  6.02579685e-02, -1.86105496e-02,
       -8.81041160e-03, -3.01994898e-02, -1.42541591e-02, -3.87884289e-02,
       -2.76125646e-02, -6.94957634e-03,  3.20653388e-02, -5.17838445e-03,
        8.37960027e-02, -8.87329694e-02, -7.30260818e-02,  5.92846727e-02,
        4.07805582e-02,  7.72684693e-02, -4.60029423e-02,  3.98426848e-02,
        1.74821532e-02,  8.70262992e-03, -3.09291007e-02,  2.20470406e-02,
        4.76479838e-02,  1.76566216e-02, -3.62013813e-02, -3.53408182e-02,
       -5.92536405e-03,  2.97091266e-02,  8.05883488e-02,  1.07132627e-02,
       -5.38192486e-02, -9.62880078e-03, -1.17263736e-01,  3.36546383e-02,
        7.05763513e-03,  7.70160054e-03, -5.17482624e-02,  5.46955475e-02,
       -7.80289880

In [17]:
print(min(results[0]))

-0.11726373551188797


## 2. Cosine similarity with another vector

The vectors that our embedding model returns are already normalized: their length is 1.0.

You can check that by using the norm function:

In [17]:
np.linalg.norm(results)

np.float64(1.0)

Which means that we can simply compute the dot product between two vectors to learn the cosine similarity between them.

For example, if you compute the cosine of the query vector with itself, the result will be 1.0:

In [20]:
print(results[0].dot(results[0]))

1.0


In [18]:
doc = 'Can I still join the course after the start date?'

# Embed the document
doc_embedding = list(embedding_model.embed([doc]))[0]

# Get the query embedding (already computed above)
query_embedding = results

# Calculate cosine similarity
def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    return dot_product / (norm_v1 * norm_v2)

# Calculate the cosine similarity
cosine_sim = cosine_similarity(query_embedding, doc_embedding)
print(f"Cosine similarity between query and document: {cosine_sim}")

Cosine similarity between query and document: [0.90085289]


In [22]:
print(results[0].dot(doc_embedding))

0.9008528856818037


## 3. Ranking by cosine




In [23]:
new_documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [29]:
query_docs = [x.get('text') for x in new_documents]

embedding_generator_2 = embedding_model.embed(query_docs)

In [30]:
embeddins = list(embedding_generator_2)

In [33]:
print([float(results[0].dot(x)) for x in embeddins])

[0.7629684493123693, 0.8182378361919107, 0.8085397290762828, 0.7133078539597724, 0.7304499528359614]


## 4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of question and text:

`full_text = doc['question'] + ' ' + doc['text']`

Embed this field and compute the cosine between it and the query vector. What's the highest scoring document?

In [36]:
text_question_docs = [x.get('question') + x.get('text') for x in new_documents]

embedding_generator_3 = embedding_model.embed(text_question_docs)

embeddins_3 = list(embedding_generator_3)

print([float(results[0].dot(x)) for x in embeddins_3])

[0.8514542983476503, 0.8436594058703555, 0.8408287206920106, 0.7755157689596764, 0.8086007942697778]


It's a different (more contextually appropriate) document because the embeddings represent more information than the previous documents embedded, because they included Q&A instead of just Q.  

If you examine the 2 choices made in the last 2 questions, a human brain says yes, those are the right documents in both cases.  

## 5. Selecting the embedding model

Now let's select a smaller embedding model. What's the smallest dimensionality for models in fastembed?

In [37]:
TextEmbedding.list_supported_models()

[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

In [42]:
from pprint import pprint

pprint([x for x in TextEmbedding.list_supported_models() if x.get('model') == 'BAAI/bge-small-en'])

[{'additional_files': [],
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens '
                 'truncation, Prefixes for queries/documents: necessary, 2023 '
                 'year.',
  'dim': 384,
  'license': 'mit',
  'model': 'BAAI/bge-small-en',
  'model_file': 'model_optimized.onnx',
  'size_in_GB': 0.13,
  'sources': {'_deprecated_tar_struct': True,
              'hf': 'Qdrant/bge-small-en',
              'url': 'https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz'},
  'tasks': {}}]


## 6. Indexing with qdrant (2 points)